# 환경셋업

In [53]:
# 패키지 설치
!pip install dserestapi 


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [3]:
# 사용자 본인의 ID/PW 를 입력하시기 바랍니다.
import json 
credential = r"N:\My Drive\__CREDENTIALS__\3DS\JLE69_GMAIL\CML Agent ID-PW.json"
with open(credential, "r", encoding='utf-8') as f:
    cred = json.load(f)
os.environ["CLM_AGENT_ID"] = cred['Agent ID']
os.environ["CLM_AGENT_PASSWORD"] = cred['Agent Password']

In [4]:
# 사용하시는 테넌트 URI 를 확인하시고 바꾸시기 바랍니다.
os.environ['3DX_PLATFORM_TENANT_URI'] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"

In [52]:
from importlib import reload 
import dserestapi 
reload(dserestapi)
from dserestapi import Storages, ObjectStorage, SemanticGraphIndex

# USE CASE - 01
- 데이터 모델링 
- SemanticGraphIndex Storage 신규 생성 
- 생성시 데이터 모델링을 동시에 수행 

In [ ]:
# 데이터 모델링 
data_modeling_config = {
    "datamodel": {
        "classes": [
            {
                "name": "DummyData",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            }
        ]
    }
}

In [10]:
# 스토리지 생성
api = Storages()
res = api.create(
    stype="IndexUnit",
    name="TestSGI_01_CreatedByRestAPI",
    description="테스트 후 삭제 | SGI 데이터 모델링 테스트",
    config=data_modeling_config
)
with open(os.path.join(os.getcwd(), "Create A SGI.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)


응답코드--> 201


In [12]:
sgi = SemanticGraphIndex()
res = sgi.get_index(sgi_name="TestSGI_01_CreatedByRestAPI")
with open(os.path.join(os.getcwd(), "Get an Index.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

sgi_info = res.json()
resource_uuid = sgi_info["resourceUUID"]
resource_uuid


응답코드--> 200


'5327b9fc-dbd7-40dc-9136-082e0cc1c6d8'

In [41]:
# 생성한 데이터 모델 개수 확인
res = sgi.class_count(resourceUUID=resource_uuid, pkg_name="Rawdata", class_name_li=["DummyData"])
with open(os.path.join(os.getcwd(), "Class Count.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)


응답코드--> 200


# USE CASE - 02
- 데이터 모델 스키마에 맞는 데이터 생성 
- 스키마에 맞는 데이터 타입인지 정합성 체크 
- 생성한 SGI의 타겟 데이터 모델에 데이터 주입/적재 

In [44]:
# 데이터 생성 
data = [
    {
        "class": "Rawdata.DummyData",
        "seq": 1,
        "title": "This is a title",
        "_text": "bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
    {
        "class": "Rawdata.DummyData",
        "seq": 2,
        "title": "This is a title-2",
        "_text": "(2) bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
]

In [45]:
# 주입할 데이터의 정합성 체크 
sgi = SemanticGraphIndex()
res = sgi.validateItemsEvent(resourceUUID=resource_uuid, action="AddOrReplaceItem", data=data)
res 
with open(os.path.join(os.getcwd(), "Validation Items.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)


응답코드--> 200


In [46]:
# 데이터 주입/적재
res = sgi.ingest(resourceUUID=resource_uuid, data=data)
with open(os.path.join(os.getcwd(), "Ingest Data.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)


응답코드--> 200


In [47]:
# 데이터 주입/적재 결과 알림 받기 
res = sgi.notification(resourceUUID=resource_uuid)
with open(os.path.join(os.getcwd(), "Notification.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)


응답코드--> 200


In [49]:
# 주입한 데이터의 object 별 uri 가져오기
res = sgi.get_uri(resourceUUID=resource_uuid)
with open(os.path.join(os.getcwd(), "SGI URI.json"), "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)


응답코드--> 200
